In [1]:
from pydrake.symbolic import Variable
from pydrake.systems.primitives import SymbolicVectorSystem

In [2]:
#@ test simple sys ---> no infrastruct
x = Variable('x')
continuos_vector_system = SymbolicVectorSystem(state=[x], dynamics=[-x  + x**3], output=[x] )
discrete_vector_system = SymbolicVectorSystem(state=[x], dynamics=[x**3], output=[x], time_period=1.0)

In [3]:
from pydrake.systems.framework import LeafSystem

In [8]:
#@ same result of symbolic sys, just here i can insert more dettails
class ContinuousTimeSystem(LeafSystem): 
    def __init__(self): 
        LeafSystem.__init__(self)
        
        state_index = self.DeclareContinuousState(1)
        self.DeclareStateOutputPort('y', state_index)
        
    def CalcTimeDerivatives(self, context, derivatives):
        # xdot(t) = -x(t) + x^3(t)
        x = context.get_continuous_state_vector().GetAtIndex(0)
        xdot = -x + x**3
        derivatives.get_mutable_vector().SetAtIndex(0, xdot)

            
class DiscreteTimeSystem(LeafSystem): 
    def __init__(self): 
        LeafSystem.__init__(self)
        
        state_index = self.DeclareDiscreteState(1)
        self.DeclareStateOutputPort("y", state_index)
        self.DeclarePeriodicDiscreteUpdateEvent(
            period_sec=1.0,
            offset_sec=0.0, #first time call
            update=self.Update
        )
    
    def Update(self, contex, discrete_state):
        x = context.get_discrete_state_vector().GetAtIndex(0)
        x_new = x**3
        discrete_state.get_mutable_vector().SetAtIndex(0, xnext)
        

In [5]:
continuos_system = ContinuousTimeSystem()
discrete_system = DiscreteTimeSystem()

In [6]:
import matplotlib.pyplot as plt
from pydrake.systems.analysis import Simulator
from pydrake.systems.framework import DiagramBuilder
from pydrake.systems.primitives import LogVectorOutput

In [10]:
%tb
builder = DiagramBuilder()
system = builder.AddSystem(ContinuousTimeSystem())
logger = LogVectorOutput(system.get_output_port(0), builder)
diagram = builder.Build()


context = diagram.CreateDefaultContext()
context.SetContinuousState([0.9])


simulator = Simulator(diagram, context)
simulator.AdvanceTo(10) #10 second

log = logger.FindLog(context)
plt.figure()
plt.plot(log.sample_times(), log.data().transpose())
plt.xlabel('t')
plt.ylabel('y(t)')

SystemExit: Failure at systems/framework/system.cc:1103 in DoCalcTimeDerivatives(): condition 'derivatives->size() == 0' failed.

SystemExit: Failure at systems/framework/system.cc:1103 in DoCalcTimeDerivatives(): condition 'derivatives->size() == 0' failed.

In [13]:
builder = DiagramBuilder()
system = builder.AddSystem(DiscreteTimeSystem())
logger = LogVectorOutput(system.get_output_port(0), builder)
diagram = builder.Build()

simulator = Simulator(diagram)

state = simulator.get_mutable_context().get_mutable_discrete_state_vector()
state.SetFromVector([0.9])

simulator.AdvanceTo(10) #10 second

log = logger.FindLog(context)
plt.figure()
plt.plot(log.sample_times(), log.data().transpose())
plt.xlabel('t')
plt.ylabel('y(t)')

RuntimeError: Cannot use DiscreteValues convenience methods unless there is exactly one group. num_groups() = 0

In [16]:
import matplotlib.pyplot as plt
import numpy as np
import pydot
from IPython.display import SVG, display
from pydrake.examples import PendulumPlant
from pydrake.systems.analysis import Simulator
from pydrake.systems.controllers import PidController
from pydrake.systems.framework import DiagramBuilder
from pydrake.systems.primitives import LogVectorOutput

builder = DiagramBuilder()

# First add the pendulum.
pendulum = builder.AddSystem(PendulumPlant())
pendulum.set_name("pendulum")

controller = builder.AddSystem(PidController(kp=[10.], ki=[1.], kd=[1.]))
controller.set_name("controller")

# Now "wire up" the controller to the plant.
builder.Connect(pendulum.get_state_output_port(), controller.get_input_port_estimated_state())
builder.Connect(controller.get_output_port_control(), pendulum.get_input_port())

# Make the desired_state input of the controller an input to the diagram.
builder.ExportInput(controller.get_input_port_desired_state())

# Log the state of the pendulum.
logger = LogVectorOutput(pendulum.get_state_output_port(), builder)
logger.set_name("logger")

diagram = builder.Build()
diagram.set_name("diagram")

display(SVG(pydot.graph_from_dot_data(
    diagram.GetGraphvizString(max_depth=2))[0].create_svg()))


FileNotFoundError: [Errno 2] "dot" not found in path.

In [18]:
pydot.graph_from_dot_data

<function pydot.graph_from_dot_data(s)>